<a href="https://colab.research.google.com/github/Gautam-Agarwal/music-generator-1/blob/main/music_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import pickle
import pandas as pd
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
import tensorflow as tf
from pathlib import Path


In [ ]:
def load_songs(path):
    # Loads MIDIs and converts them into a list of sequence of notes
    songs = []
    folder = Path(path)
    for file in folder.rglob('*.mid'):
        songs.append(file)

    return songs

In [ ]:
def preprocess_songs(songs):
    notes = []
    for i,file in enumerate(songs):
        print(f'{i+1}: {file}')
        try:
            midi = converter.parse(file)
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)
            if parts: # file has instrument parts
                notes_to_parse = parts.parts[0].recurse()
            else: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except:
            print(f'FAILED: {i+1}: {file}')


    # Save notes to Drive for future usage
    with open('notes', 'wb') as filepath:
        pickle.dump(notes, filepath)



In [ ]:
path = '/content/drive/MyDrive/myMidi'
songs = load_songs(path)
preprocess_songs(songs)

1: /content/drive/MyDrive/myMidi/TRAP EDM.mid
2: /content/drive/MyDrive/myMidi/drum1.mid
3: /content/drive/MyDrive/myMidi/Piano Piece Bridge.mid
4: /content/drive/MyDrive/myMidi/Progression based.mid
5: /content/drive/MyDrive/myMidi/C Major Wonder.logixc.mid
6: /content/drive/MyDrive/myMidi/D major chord progression.mid
7: /content/drive/MyDrive/myMidi/pop.mid
8: /content/drive/MyDrive/myMidi/halftime.mid
FAILED: 8: /content/drive/MyDrive/myMidi/halftime.mid
9: /content/drive/MyDrive/myMidi/Final Comp.mid
10: /content/drive/MyDrive/myMidi/Avenue.mid
11: /content/drive/MyDrive/myMidi/Jazz.mid
12: /content/drive/MyDrive/myMidi/Ballad.mid
13: /content/drive/MyDrive/myMidi/horn chord beat.mid
14: /content/drive/MyDrive/myMidi/beginning.mid
15: /content/drive/MyDrive/myMidi/Rock.mid
16: /content/drive/MyDrive/myMidi/RNB 2.mid
17: /content/drive/MyDrive/myMidi/idk.mid
18: /content/drive/MyDrive/myMidi/orchestral.mid


In [ ]:
def load_notes():
    with open('notes','rb') as filepath:
        notes = pickle.load(filepath)

    return notes

In [ ]:
def preocess_notes(notes):

    dataSize = 150000
    allNotes = notes
    allNotes = allNotes[:dataSize]

    # Look at 10 previous notes to make a prediction
    #   We can tune this parameter if needed, based on the length of 
    #   chord progressions
    seqLength = 10
    print('Using sequence length of {}'.format(seqLength))

    pitchSet = sorted(set(allNotes))
    numPitches = len(pitchSet)
    # here pitches are either notes or chords
    #   they are sorted lexicographically, so a chord 'C4.E4' will come after a
    #   note 'C4'
    print('Identified {} pitches'.format(numPitches))

    # Map each note/chord to a number normalized to (0,1)
    pitchMapping = dict((note, number) for (number, note) in enumerate(pitchSet))

    networkInput = []
    networkOutput = []

    print('Starting sequencing of {} notes'.format(len(allNotes)))
    for i in range(0, len(allNotes)- seqLength):
        sequenceIn = allNotes[i:i+seqLength]
        predictionOut = allNotes[i+seqLength]

        networkInput.append([pitchMapping[note] for note in sequenceIn])
        networkOutput.append(pitchMapping[predictionOut])

        if (i+1) % 400000 == 0:
            print('Finished making {} sequences'.format(i+1))

    networkInput = np.array(networkInput)
    networkOutput = np.array(networkOutput)

    numSeqs = len(networkInput)
    # reshape input to match the LSTM layer format
    networkInputShaped = np.reshape(networkInput, (numSeqs, seqLength, 1))
    networkInputShaped = networkInputShaped / numPitches

    networkOutputShaped = np_utils.to_categorical(networkOutput)

    return networkInput, networkOutput, networkInputShaped, networkOutputShaped, numPitches


In [ ]:
def create_model(networkInputShaped,networkOutputShaped,numPitches,num_epochs=30):
    model = Sequential()
    model.add(Dropout(0.2))
    model.add(LSTM(
        512,
        input_shape=(networkInputShaped.shape[1], networkInputShaped.shape[2]),
        return_sequences=True
    ))
    model.add(Dense(256))
    model.add(Dense(256))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dense(256))
    model.add(LSTM(512))
    model.add(Dense(numPitches))
    model.add(Dense(numPitches))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    num_epochs = 30

    filepath = "/weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger_1.hdf5"
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', 
        verbose=1,        
        save_best_only=True,        
        mode='min'
    )    
    callbacks_list = [checkpoint]


    history = model.fit(networkInputShaped, networkOutputShaped, epochs=num_epochs, batch_size=64, callbacks=callbacks_list)

    return model, history


In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    # Selects a random row from the network_input
    start = np.random.randint(0, len(network_input)-1)
    print(f'start: {start}')
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        print(note_index)
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern,index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='output.mid')


In [ ]:
notes = load_notes()

n_vocab = len(set(notes))
print(n_vocab)
pitchnames = sorted(set(item for item in notes))

networkInput, networkOutput, networkInputShaped, networkOutputShaped, numPitches = preocess_notes(notes)
model, history = create_model(networkInputShaped,networkOutputShaped,numPitches,num_epochs=30)
prediction_output = generate_notes(model, networkInputShaped, pitchnames, n_vocab)

create_midi(prediction_output)

169
Using sequence length of 10
Identified 169 pitches
Starting sequencing of 30786 notes
Epoch 1/30
481/481 [==============================] - ETA: 0s - loss: 4.2354 - accuracy: 0.0820
Epoch 1: loss improved from inf to 4.23540, saving model to /weights/weights-improvement-01-4.2354-bigger_1.hdf5
481/481 [==============================] - 369s 748ms/step - loss: 4.2354 - accuracy: 0.0820
Epoch 2/30
481/481 [==============================] - ETA: 0s - loss: 4.0140 - accuracy: 0.0987
Epoch 2: loss improved from 4.23540 to 4.01397, saving model to /weights/weights-improvement-02-4.0140-bigger_1.hdf5
481/481 [==============================] - 352s 732ms/step - loss: 4.0140 - accuracy: 0.0987
Epoch 3/30
481/481 [==============================] - ETA: 0s - loss: 3.6805 - accuracy: 0.1565
Epoch 3: loss improved from 4.01397 to 3.68049, saving model to /weights/weights-improvement-03-3.6805-bigger_1.hdf5
481/481 [==============================] - 353s 734ms/step - loss: 3.6805 - accuracy: 0.1